In [2]:
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df = pd.read_csv('final325.csv')


Pivoting the data - from  wide format to long format for ease of analysis

In [4]:
import pandas as pd

value_vars = [
    'ESG-FY0', 'ESG-FY1', 'ESG-FY2', 'ESG-FY3', 'ESG-FY4',
    'SP-FY0', 'SP-FY1', 'SP-FY2', 'SP-FY3', 'SP-FY4',
    'GP-FY0', 'GP-FY1', 'GP-FY2', 'GP-FY3', 'GP-FY4',
    'EP-FY0', 'EP-FY1', 'EP-FY2', 'EP-FY3', 'EP-FY4',
    'Tobin Q-FY0', 'Tobin Q-FY1', 'Tobin Q-FY2', 'Tobin Q-FY3', 'Tobin Q-FY4',
    'Growth-FY0', 'Growth-FY1', 'Growth-FY2', 'Growth-FY3', 'Growth-FY4',
    'CashFlow-FY0', 'CashFlow-FY1', 'CashFlow-FY2', 'CashFlow-FY3', 'CashFlow-FY4',
    'Liquidity-FY0', 'Liquidity-FY1', 'Liquidity-FY2', 'Liquidity-FY3', 'Liquidity-FY4',
    'Total Assets Reported-FY0', 'Total Assets Reported-FY1', 'Total Assets Reported-FY2', 'Total Assets Reported-FY3', 'Total Assets Reported-FY4',
    'Profitability-FY0', 'Profitability-FY1', 'Profitability-FY2', 'Profitability-FY3', 'Profitability-FY4',
    'Debt-FY0', 'Debt-FY1', 'Debt-FY2', 'Debt-FY3', 'Debt-FY4',
    'leverage-FY0', 'leverage-FY1', 'leverage-FY2', 'leverage-FY3', 'leverage-FY4',
    'Net Income-FY0','Net Income-FY1','Net Income-FY2','Net Income-FY3','Net Income-FY4', 
]

df_long = df.melt(id_vars=['Identifier', 'GICS Sector Name'], value_vars=value_vars, var_name='Variable', value_name='Value')

df_long['FiscalYear'] = df_long['Variable'].str.extract(r'FY(\d)').astype(int)
df_long['VariableType'] = df_long['Variable'].str.extract(r'([A-Za-z ]+)-FY\d')

df_pivot = df_long.pivot_table(index=['Identifier', 'FiscalYear', 'GICS Sector Name'], columns='VariableType', values='Value', aggfunc='first')

df_pivot.reset_index(inplace=True)

print(df_pivot.head())


VariableType Identifier  FiscalYear GICS Sector Name  CashFlow      Debt  \
0                   A.N           0      Health Care  9.179264  9.436957   
1                   A.N           1      Health Care  9.196176  9.436640   
2                   A.N           2      Health Care  9.184975  9.436004   
3                   A.N           3      Health Care  9.011570  9.358696   
4                   A.N           4      Health Care  9.117271  9.253096   

VariableType         EP        ESG         GP     Growth  Liquidity  \
0             80.121915  81.937689  67.679398  10.646474  -4.175048   
1             79.752529  87.656856  87.108600  10.683177  -4.203332   
2             79.447415  86.658778  81.009410  10.560607  -3.228584   
3             79.733560  87.855639  83.406537  10.422619  -3.453879   
4             77.953547  89.330448  88.244949  10.332201  -3.829046   

VariableType  Net Income  Profitability         SP   Tobin Q  \
0               9.093422       0.115210  93.410336  

In [ ]:
df_pivot.to_excel('pivot_final.xlsx', index=False)

VIF and Breuch Pagan test

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare

df.set_index(['Identifier', 'FiscalYear'], inplace=True)

df['ESGIND_CompAdvantage'] = df['EP'] * df['Net Income']
df['ESGSCORE_CompAdvantage'] = df['ESG'] * df['Net Income']

control_vars = [ 'Growth', 'Profitability', 'Liquidity', 'Cashflow', 'Dedbt', 'Total Assets Reported']

dependent_var = 'Tobin Q'

exog_vars_model_1 = ['EP', 'Net Income'] + control_vars
exog_vars_model_2 = ['EP', 'Net Income', 'ESGIND_CompAdvantage'] + control_vars
exog_vars_model_3 = ['EP', 'Net Income'] + control_vars
exog_vars_model_4 = ['EP', 'CompA', 'ESGSCORE_CompAdvantage'] + control_vars

df['constant'] = 1

# Models
models = {
    'Model 1': exog_vars_model_1,
    'Model 2': exog_vars_model_2,
    'Model 3': exog_vars_model_3,
    'Model 4': exog_vars_model_4
}

results = {}
for model_name, exog_vars in models.items():
    mod = PanelOLS(df[dependent_var], df[exog_vars])
    res = mod.fit(cov_type='clustered', cluster_entity=True)
    results[model_name] = res


model_comparison = compare(results)


model_comparison.summary


#VIF
def calculate_vif(X, exog_vars):
    X = sm.add_constant(X[exog_vars])
    
    vif_data = pd.DataFrame()
    vif_data['variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_data = calculate_vif(df, exog_vars_model_4)

# Breusch-Pagan test 
model_for_test = results['Model 1']
bp_test = het_breuschpagan(model_for_test.resids, model_for_test.model.exog)
bp_test_results = dict(zip(['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'], bp_test))

vif_data, bp_test_results


4 models

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare


# Since the data is already in long format, we assume 'Identifier' is the firm ID and 'FiscalYear' is the time ID
df.set_index(['Identifier', 'FiscalYear'], inplace=True)


# interaction terms for the models 
df['ESGIND_CompAdvantage'] = df['EP'] * df['Net Income']
df['ESGSCORE_CompAdvantage'] = df['ESG'] * df['Net Income']

# Define control variables
control_vars = [ 'Growth', 'Profitability', 'Liquidity', 'CashFlow', 'Debt', 'Total Assets Reported']

# Dependent variable
dependent_var = 'Tobin Q'

# Model specifications 
exog_vars_model_1 = ['EP', 'Net Income'] + control_vars
exog_vars_model_2 = ['EP', 'Net Income', 'ESGIND_CompAdvantage'] + control_vars
exog_vars_model_3 = ['ESG', 'Net Income'] + control_vars
exog_vars_model_4 = ['ESG', 'Net Income', 'ESGSCORE_CompAdvantage'] + control_vars

# Adding a constant to the models
df['constant'] = 1

# Models
models = {
    'Model 1': exog_vars_model_1,
    'Model 2': exog_vars_model_2,
    'Model 3': exog_vars_model_3,
    'Model 4': exog_vars_model_4
}

# Dictionary to store results
results = {}

# Loop through the models and run PanelOLS
for model_name, exog_vars in models.items():
    mod = PanelOLS(df[dependent_var], df[exog_vars])
    res = mod.fit(cov_type='clustered', cluster_entity=True)
    results[model_name] = res

# Compare models
model_comparison = compare(results)

# Display model comparison
model_comparison.summary